In [ ]:
from IPython.core.display import display, HTML

def highlight_corrected_tokens(response, corrected_tokens, tokenizer):
    """
    使用 HTML 高亮被 target model 修正的 token（红色背景）。
    这里保证 token 级别的一一对应。
    """
    # **使用 tokenizer 进行 token 级别拆分**
    token_ids = tokenizer(response, return_tensors="pt").input_ids[0].tolist()
    tokens = [tokenizer.decode([tid]) for tid in token_ids]  # 确保 token 与 token_id 对应

    highlighted_text = []
    
    for i, token in enumerate(tokens):
        # 检查该位置是否在 corrected_tokens 中
        correction = next((c for c in corrected_tokens if c["position"] == i), None)
        if correction:
            highlighted_text.append(f'<span style="background-color: red; color: white;">{correction["target"]}</span>')
        else:
            highlighted_text.append(token)

    return " ".join(highlighted_text)

def display_results(results, tokenizer):
    """
    在 Notebook 中展示问题、生成的答案，并高亮被修正的 token。
    """
    for item in results:
        question = item["question"]
        generated_answer = item["generated_answer"]
        corrected_tokens = item["corrected_tokens"]

        highlighted_answer = highlight_corrected_tokens(generated_answer, corrected_tokens, tokenizer)

        html_content = f"""
        <div style="border: 1px solid #ccc; padding: 10px; margin-bottom: 10px;">
            <p><b>Question:</b> {question}</p>
            <p><b>Generated Answer:</b> {highlighted_answer}</p>
        </div>
        """
        display(HTML(html_content))

# **加载 JSON 结果**
import json
from transformers import AutoTokenizer

with open("math500_results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# **加载 tokenizer，确保和生成时一致**
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# **显示结果**
display_results(results, tokenizer)